In [123]:
import Pkg;
Pkg.add("PyPlot")
using PyPlot
using Random, LinearAlgebra, Statistics, DataFrames, Plots
Pkg.add("CSV")
using CSV

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [124]:
using LowRankModels

In [125]:
data = CSV.read("weather.csv")
head(data)

,dae,max_emperaure_f,mean_emperaure_f,min_emperaure_f,max_dew_poin_f,mean_dew_poin_f
,String,Int64?,Int64?,Int64?,Int64?,Int64?
1,8/29/13,74,68,61,61,58
2,8/30/13,78,69,60,61,58
3,8/31/13,71,64,57,57,56
4,9/1/13,74,66,58,60,56
5,9/2/13,75,69,62,61,60
6,9/3/13,73,67,60,59,56


In [126]:
# Look at the column names
feature_names = names(data)
for i in 1:size(data)[2]
    println(string(i), "\t", string(feature_names[i]), "\t", string(eltype(data[!, i])))
end

1	dae	String
2	max_emperaure_f	Union{Missing, Int64}
3	mean_emperaure_f	Union{Missing, Int64}
4	min_emperaure_f	Union{Missing, Int64}
5	max_dew_poin_f	Union{Missing, Int64}
6	mean_dew_poin_f	Union{Missing, Int64}
7	min_dew_poin_f	Union{Missing, Int64}
8	max_humidiy	Union{Missing, Int64}
9	mean_humidiy	Union{Missing, Int64}
10	min_humidiy	Union{Missing, Int64}
11	max_sea_level_pressure_inches	Union{Missing, Float64}
12	mean_sea_level_pressure_inches	Union{Missing, Float64}
13	min_sea_level_pressure_inches	Union{Missing, Float64}
14	max_visibiliy_miles	Union{Missing, Int64}
15	mean_visibiliy_miles	Union{Missing, Int64}
16	min_visibiliy_miles	Union{Missing, Int64}
17	max_wind_Speed_mph	Union{Missing, Int64}
18	mean_wind_speed_mph	Union{Missing, Int64}
19	max_gus_speed_mph	Union{Missing, Int64}
20	precipiaion_inches	Union{Missing, Float64}
21	cloud_cover	Union{Missing, Int64}
22	evens	Union{Missing, String}
23	wind_dir_degrees	Union{Missing, Int64}
24	zip_code	Int64


In [127]:
# inserts "None" for every blank cell in Events 
# data.events = coalesce.(data.events, "None")

# TODO: delete row 2912 (since that row is completely empty: 8/8/15)

In [128]:
# locate missing data in the precipitation_inches column (column #20)
missingList = []

for row in 1:size(data)[1]
    if ismissing(data[row,20])
        push!(missingList, (20,row))
    end
end

# in total there are 143 missing elements that come from:
# cols 20
# rows 33, 60, 76 ... , 3653

missingList

144-element Array{Any,1}:
 (20, 33)
 (20, 60)
 (20, 76)
 (20, 85)
 (20, 96)
 (20, 132)
 (20, 134)
 (20, 136)
 (20, 154)
 (20, 155)
 (20, 166)
 (20, 171)
 (20, 373)
 ⋮
 (20, 3525)
 (20, 3536)
 (20, 3561)
 (20, 3564)
 (20, 3582)
 (20, 3604)
 (20, 3606)
 (20, 3612)
 (20, 3613)
 (20, 3622)
 (20, 3634)
 (20, 3653)

In [129]:
# impute missing data
# losses = Loss[QuadLoss(), LogisticLoss(), HingeLoss()]
# rw = Regularizer[QuadReg(), QuadReg(), OneReg()]

data = data[:,2:21]
data

,max_emperaure_f,mean_emperaure_f,min_emperaure_f,max_dew_poin_f,mean_dew_poin_f
,Int64?,Int64?,Int64?,Int64?,Int64?
1,74,68,61,61,58
2,78,69,60,61,58
3,71,64,57,57,56
4,74,66,58,60,56
5,75,69,62,61,60
6,73,67,60,59,56
7,74,68,61,59,57
8,72,66,60,57,56
9,85,71,56,57,51


In [130]:
# impute missing data
# losses = Loss[QuadLoss(), LogisticLoss(), HingeLoss()]
# rw = Regularizer[QuadReg(), QuadReg(), OneReg()]

Y = data
l2 = QuadLoss()
l1 = L1Loss()
nonneg = NonNegConstraint()
quad = QuadReg()
lin = OneReg()
zero = ZeroReg()
k = size(data)[2]

Ω = [(x,y) for x=1:size(data)[1] , y=1:size(data)[2] if !isMissing(data[x,y])];

In [131]:
glrm = GLRM(Y, l1, quad, quad, k, obs=Ω)
X,W,ch = fit!(glrm, maxiter=1000)

Fitting GLRM
Iteration 10: objective value = 238901.48636763904
Iteration 20: objective value = 166522.61425776224
Iteration 30: objective value = 131711.63649227258
Iteration 40: objective value = 113077.13373873197
Iteration 50: objective value = 102001.5275460532
Iteration 60: objective value = 95523.95843358946
Iteration 70: objective value = 91486.05446110108
Iteration 80: objective value = 88822.38566641975
Iteration 90: objective value = 86944.01443145782
Iteration 100: objective value = 85426.50336860861


([-0.04898225743654448 -0.47000857403869434 … -1.063783284619916 -0.8109767842778177; -0.6569391969206061 -0.7427771563123813 … -1.45619300128584 -1.4946002823316897; … ; -0.7598351158645583 -0.7251606405389017 … -0.41259627036987284 0.16597609690879522; 0.5851168473497012 0.14587626013361638 … 0.2482117212302724 0.3389742922599046], [-4.456830694709589 -3.578187616028116 … 0.14428246375156978 0.5896452700660914; -9.286000229844388 -7.428950605700337 … -0.30650648549348897 0.41800659023840303; … ; -1.194436998647146 -2.0207051491036143 … -0.324843706871191 -0.6661999072910036; 0.6096340970449956 2.2143562511890797 … 0.14705474936823454 -0.5090240752021588], ConvergenceHistory("ProxGradGLRM", [2.608133676395894e6, 2.445736035337682e6, 2.0842395628861052e6, 928360.4738207563, 530360.1388675608, 412146.9307775741, 352385.02248461347, 303046.86457144003, 275003.26169041835, 251404.47246990705  …  86769.23015978294, 86595.92886876146, 86435.21441535352, 86282.53466496497, 86133.0543773437, 

In [132]:
imputed_data = impute_missing(glrm)

3665×20 Array{Number,2}:
 74  68  61  61  58  56  93  75  57  30.07  …  10  10  10  23  11  28  0.0  4
 78  69  60  61  58  56  90  70  50  30.05     10  10   7  29  13  35  0.0  2
 71  64  57  57  56  54  93  75  57  30.0      10  10  10  26  15  31  0.0  4
 74  66  58  60  56  53  87  68  49  29.96     10  10  10  25  13  29  0.0  4
 75  69  62  61  60  58  93  77  61  29.97     10  10   6  23  12  30  0.0  6
 73  67  60  59  56  51  84  65  46  30.02  …  10  10  10  24  15  31  0.0  2
 74  68  61  59  57  56  90  72  53  30.05     10  10  10  29  19  35  0.0  4
 72  66  60  57  56  54  90  74  57  30.04     10  10  10  31  21  37  0.0  3
 85  71  56  57  51  45  86  58  29  30.0      10  10  10  24   8  28  0.0  0
 88  73  58  64  54  46  86  59  31  29.88     10  10  10  21   8  25  0.0  1
 74  65  56  58  54  52  86  70  53  29.88  …  10  10  10  26  11  31  0.0  2
 76  66  55  58  55  52  90  70  50  29.82     10  10  10  26  11  30  0.0  1
 74  66  57  59  56  54  93  73  53  29